In [1]:
import pandas as pd
import numpy as np

from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag


## Sentence Average WordNet Distance

[This paper]((https://www.aaai.org/Papers/AAAI/2006/AAAI06-123.pdf) describes a method for scoring the sematic difference of sentences using WordNet. 

The idea is a scoring scheme which looks at the average "distance" of the words in the two sentences.

To calculate the distance between sentences $T_1$ and $T_2$, go through the words in $T_1$ and find the word which is "closest" to it in $T_2$. Average these distances across all the words in $T_1$. 

To get a symmetric scoring scheme (i.e $Similiarity(T_1,T_2) = Similarity(T_2,T_1)$, perform the same operation, but this time starting with $T_2$ and findint the most similar words in $T_1$ to each word. The paper also suggests the use of _IDF_ scores.

The score can be summarized: (Mihalcea, Corley, Strapparava)

$$
Similarity(T_1,T_2) = \frac{1}{2}( \frac{\sum_{w \in T_1}MaxSimilarity(w, T_2)idf(w)}{\sum_{w \in T_1}idf(w)}
+ \frac{\sum_{w \in T_2}MaxSimilarity(w, T_1)idf(w)}{\sum_{w \in T_2}idf(w)})
$$

Where $Similarity(w, T_i)$ is the greatest similarity between a word $w$ and word in $T_i$, according to some similarity metric (with respect to WordNet). This is a heuristic for getting the semantic similarity of sentences. 

Note: word comparisons are limited to words of the same part of speech. Furthermore, each word is always assumed to be the most common sense of that word. 

As a start, I'll implement this idea without _IDF_ scores, using a variety of WordNet distance metrics. If time permits, I'll include Idf scores as well. 

**Note**: Much of this code is taken from [this great blog post!](http://nlpforhackers.io/wordnet-sentence-similarity/)

### 0. Load data to experiment with

In [21]:
data = pd.read_csv("../data/processed/train.csv")

In [22]:
data_sample = data.sample(100)

In [6]:
data_sample.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
343027,343028,343029,471095,471096,Topics for presentation ?,What is the best topic for presentation in Eng...,0
180801,180802,180802,277111,277112,What is a brief summary of Marilynne Robinson'...,What does one mean by might as well create an ...,0
194338,194339,194339,294518,294519,What is the difference between the Oxford scho...,Where is Oxford Public School ?,0
303182,303183,303184,426289,426290,Could rulers have rules in governing ?,"Why is Super Mario Maker such a good , success...",0
295076,295077,295078,105925,60263,How can I loss 50 pounds of body weight in a h...,The best way for weight loss ?,1


### 1. Simple POS tagging

In the proposed algorithm, we should only compare words of the same POS category (nouns, verbs, adjectives or adverbs). 

Below is a function which produces these broad tags for tokenized words

#### 1.0 Conert Penn POS tags to broad tags

`pos_tag` by NLTK automatcially gives fine POS tags. Example

In [14]:
pos_tag(word_tokenize("this is a pretty sentence"))

[('this', 'DT'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('pretty', 'JJ'),
 ('sentence', 'NN')]

This function brings them to more simple POS tags (courtesy of [bogdani](http://nlpforhackers.io/author/bogdani/))

In [4]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None

In [13]:
# test conversion
[(word, penn_to_wn(tag)) for (word,tag) in pos_tag(word_tokenize("this is a pretty sentence."))]

[('this', None),
 ('is', 'v'),
 ('a', None),
 ('pretty', 'a'),
 ('sentence', 'n'),
 ('.', None)]

Notice: only nounds, verbs, adjectives and adverbs are tagged. Here, the determiners and punctuations are tagged as `None`. 

### 2. Get synsets

Now we can get the synset of the most common sense of word, filtered by part of speech (again courtesy of [bogdani](http://nlpforhackers.io/author/bogdani/))

In [5]:
# get the most common synset for a tagged word from WordNet. 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        # get the first synset of the word in WordNet
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

In [24]:
# example
ex = pos_tag(word_tokenize("this is a pretty sentence."))
ex

[('this', 'DT'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('pretty', 'JJ'),
 ('sentence', 'NN'),
 ('.', '.')]

In [25]:
[tagged_to_synset(word,tag) for (word,tag) in ex]

[None,
 Synset('be.v.01'),
 None,
 Synset('pretty.s.01'),
 Synset('sentence.n.01'),
 None]

### 3. Sentence similarities

Now, I'll define series of functions for finding the sentence similarity scores for pairs of sentences, using a variety of wordnet distance metrics. 



#### 3.0 Path similarity

This score is the distance betweeen synsets in WordNet - through the hyponym/hypernym path

In [2]:
def path_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score1, count1, score2, count2 = 0.0, 0, 0.0, 0 
 
    # For each word in the first sentence
    for synset in synsets1:
        # isolate the part of speech 
        synset_pos = synset.pos()
        # Get the similarity value of the most similar word in the other sentence
        best_score = max([synset.path_similarity(ss) for ss in synsets2 if ss.pos() == synset_pos])
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score1 += best_score
            count1 += 1
            
    for synset in synsets1:
        try:
            # isolate the part of speech 
            synset_pos = synset.pos()
            # Get the similarity value of the most similar word in the other sentence
            best_score = max([synset.path_similarity(ss) for ss in synsets1 if ss.pos() == synset_pos])
        except:
            best_score = None
        # Check that the similarity could have been computed
        if best_score is not None:
            score2 += best_score
            count2 += 1
 
    # Average the values and add score from both sides to get symmetic distance
    score = .5*(score1/count1 + score2/count2)
    return(score)

In [6]:
path_similarity("The Mona-Lisa is pretty overrated if you ask me", \
                      "I hope that it will not rain tomorrow")

0.5932539682539683

In [80]:
path_similarity("Cats are beautiful animals.", "Dogs are awesome creatures")

0.8666666666666667

In [73]:
path_similarity("Trump reverses ban on importing remains of African elephants killed as trophies", \
                      "Native American tribe bracing for Keystone pipeline leak impact")

0.584054834054834

#### 3.1 Leacock-Chodorow Similarity

This similarity score is similar to the path similarity, but also incorporates the depth of the word in the WordNet taxonomy. The deeper the taxonomy depth, the higher the similarity score.

The intuition is that the deeper the word are in the tree, the longer the paths between them in genereal. The second factor counters this fact. 

In [7]:
def lch_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score1, count1, score2, count2 = 0.0, 0, 0.0, 0 
 
    # For each word in the first sentence
    for synset in synsets1:
        # isolate the part of speech 
        synset_pos = synset.pos()
        try:
            # Get the similarity value of the most similar word in the other sentence
            best_score = max([synset.lch_similarity(ss) for ss in synsets2 if ss.pos() == synset_pos])
        except:
            best_score = None
        # Check that the similarity could have been computed
        if best_score is not None:
            score1 += best_score
            count1 += 1
            
    for synset in synsets2:
        # isolate the part of speech 
        synset_pos = synset.pos()
        try:
            # Get the similarity value of the most similar word in the other sentence
            best_score = max([synset.lch_similarity(ss) for ss in synsets1 if ss.pos() == synset_pos])
        except:
            best_score = None
            
        # Check that the similarity could have been computed
        if best_score is not None:
            score2 += best_score
            count2 += 1
 
    # Average the values and add score from both sides to get symmetic score
    score = .5*(score1/count1 + score2/count2)
    return(score)

In [8]:
lch_similarity("Trump reverses ban on importing remains of African elephants killed as trophies", \
                      "Native American tribe bracing for Keystone pipeline leak impact")

1.42868502671942

#### 3.2 Wu-Palmer Similarity

This similarity score is "based on the depth of the two senses in the taxonomy and that of their Least Common Subsumer (most specific ancestor node)."

In [9]:
def wup_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score1, count1, score2, count2 = 0.0, 0, 0.0, 0 
 
    # For each word in the first sentence
    for synset in synsets1:
        try:
            # isolate the part of speech 
            synset_pos = synset.pos()
            # Get the similarity value of the most similar word in the other sentence
            best_score = max([synset.wup_similarity(ss) for ss in synsets2 if ss.pos() == synset_pos])
        except:
            best_score = None
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score1 += best_score
            count1 += 1
            
    for synset in synsets1:
        try:
            # isolate the part of speech 
            synset_pos = synset.pos()
            # Get the similarity value of the most similar word in the other sentence
            best_score = max([synset.wup_similarity(ss) for ss in synsets1 if ss.pos() == synset_pos])
        except:
            best_score = None
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score2 += best_score
            count2 += 1
 
    # Average the values and add score from both sides to get symmetic distance
    score = .5*(score1/count1 + score2/count2)
    return(score)

In [81]:
wup_similarity("Cats are beautiful animals.", "Dogs are awesome creatures")

0.9761904761904763

In [10]:
wup_similarity("The Mona-Lisa is pretty overrated if you ask me", \
                      "I hope that it will not rain tomorrow")

0.655952380952381

In [11]:
wup_similarity("Trump reverses ban on importing remains of African elephants killed as trophies", \
                      "Native American tribe bracing for Keystone pipeline leak impact")

0.6760551948051948

In [17]:
path_similarity("I like to study in the montreal area with my friends", \
                      "I like to study in New York with my buddies")

0.8263888888888888

In [26]:
data_sample.apply(lambda x : wup_similarity(x['question1'],x['question2']), axis = 1)


145691    0.774854
328960    0.839566
56745     0.793651
201812    0.958333
116988    0.933333
121810    0.966667
364092    0.780769
253334    0.919298
211479    0.876374
35407     0.940000
3173      1.000000
254688    0.928571
12667     0.944444
385782    0.955357
232077    0.776313
164801    0.950000
356156    1.000000
395874    0.940000
330119    0.758303
254893    0.740957
361111    0.983333
36251     0.860043
339525    0.932692
13258     0.955556
207535    0.937500
232508    0.802778
41162     0.920192
157675    1.000000
35167     1.000000
179388    0.723334
            ...   
13550     0.642857
146950    0.657862
305762    1.000000
401878    0.875000
30538     0.929167
185019    0.962500
322550    0.864286
356529    0.975000
202260    0.794762
169098    1.000000
174406    0.797815
149426    0.704637
166594    0.925000
320309    0.920139
214412    0.814216
307781    0.944444
331025    0.822222
366586    0.909057
204536    0.942989
249814    0.725000
278501    1.000000
227814    0.

### 